# Installing Required Libraries

In [ ]:
!pip install --q crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 langchain_groq

# Importing Libraries and Warning Control

In [ ]:
from crewai import Agent, Task, Crew
from langchain_groq import ChatGroq

# Warning control
import warnings
warnings.filterwarnings('ignore')

# Defining the LLM (Language Model) & Serper Tool:

In [ ]:
llm = ChatGroq(temperature=0,
             model_name="llama3-8b-8192",
            api_key='You should write your API KEY here.')
import os
# Serper API anahtarını doğrudan çevresel değişken olarak ayarlayın
os.environ["SERPER_API_KEY"] = "You should write your SERPER API KEY here."

# Creating Tools

**crewAI Tools**

In [ ]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

# Creating Agents

In [ ]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    llm = llm,
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    )
)

In [ ]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    llm = llm,
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    )
)

In [ ]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    llm = llm,
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    )
)

# Creating Venue Pydantic Object

*   Create a class VenueDetails using Pydantic BaseModel.
*   Agents will populate this object with information about different venues by creating different instances of it.

In [ ]:
from pydantic import BaseModel
# Define a Pydantic model for venue details
# (demonstrating Output as Pydantic)
class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

# Creating Tasks

*   By using output_json, you can specify the structure of the output you want.
*   By using output_file, you can get your output in a file.
*   By setting human_input=True, the task will ask for human feedback (whether you like the results or not) before finalising it.
*   By setting async_execution=True, it means the task can run in parallel with the tasks which come after it.

In [ ]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",
      # Outputs the venue details as a JSON file
    agent=venue_coordinator
)

In [ ]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager
)

In [ ]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent
)

# Creating the Crew

*   **Note:** Since you set async_execution=True for logistics_task and marketing_task tasks, now the order for them does not matter in the tasks list.

In [ ]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator,
            logistics_manager,
            marketing_communications_agent],

    tasks=[venue_task,
           logistics_task,
           marketing_task],

    verbose=True
)

# Running the Crew

In [ ]:
event_details = {
    'event_topic': "Tech Innovation Conference",
    'event_description': "A gathering of tech innovators "
                         "and industry leaders "
                         "to explore future technologies.",
    'event_city': "San Francisco",
    'tentative_date': "2024-09-15",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Conference Hall"
}

In [ ]:
result = event_management_crew.kickoff(inputs=event_details)

 [DEBUG]: == Working Agent: Venue Coordinator
 [INFO]: == Starting Task: Find a venue in San Francisco that meets criteria for Tech Innovation Conference.


> Entering new CrewAgentExecutor chain...
Thought: I need to start by gathering information about the event and the criteria for the venue.

Action: Search the internet

Action Input: {"search_query": "tech innovation conference san francisco venue"}
 


Search results: Title: World Agri-Tech Innovation Summit, March 11-12, 2025
Link: https://worldagritechusa.com/
Snippet: The annual meeting place for the global agtech ecosystem to exchange insights, be inspired, and identify future partners.
---
Title: Top 12 tech conferences in San Francisco for 2024 - TravelPerk
Link: https://www.travelperk.com/blog/top-tech-conferences-in-san-francisco/
Snippet: With over 200 speakers, the San Francisco Tech Summit offers an action-packed agenda for inspiration, insights, innovation, and connections.
---
Title: Tech Summit Silicon Valley
Link: 

**Display the generated venue_details.json file.**

In [ ]:
import json
from pprint import pprint

with open('venue_details.json') as f:
   data = json.load(f)

pprint(data)



> Entering new CrewAgentExecutor chain...
{'address': '1290 Sutter St, San Francisco, CA 94109',
 'booking_status': 'Available in March 2025, Fits within our budget',
 'capacity': 500,
 'name': 'The Regency Center'}


# Display the results of your execution as markdown.

**Display the generated marketing_report.md file.**

In [ ]:
from IPython.display import Markdown
Markdown("marketing_report.md")

marketing_report.md